# Read and analyze data in chunks.


- Download ows-raw.txt
- Read data as described in https://chengjun.github.io/mybook/06-data-cleaning-preprocessing.html Read data as chunk

In [1]:
file_path = 'E:/2024计算社会科学夏校/data/ows-raw.txt'

In [2]:
import pandas as pd

In [3]:
f = open(file_path, 'r', encoding='utf-8')
reader = pd.read_table(f,  sep=',',  quotechar='"', iterator=True, on_bad_lines='skip') #跳过报错行
chunkSize = 10**8
chunk = reader.get_chunk(chunkSize)
len(chunk)

6602120

In [4]:
chunk.head()

,Twitter ID,Text,Profile Image URL,Day,Hour,Minute,Created At,Geo,From User,From User ID,Language,To User,To User ID,Source
0,121813144174727168,RT @AnonKitsu: ALERT!!!!!!!!!!COPS ARE KETTLIN...,http://a2.twimg.com/profile_images/1539375713/...,2011-10-06,5,4,2011-10-06 05:04:51,N;,Anonops_Cop,401240477,en,NaN,0,&lt;a href=&quot;http://twitter.com/&quot;&gt;...
1,121813146137657344,@jamiekilstein @allisonkilkenny Interesting in...,http://a2.twimg.com/profile_images/1574715503/...,2011-10-06,5,4,2011-10-06 05:04:51,N;,KittyHybrid,34532053,en,jamiekilstein,2149053,&lt;a href=&quot;http://twitter.com/&quot;&gt;...
2,121813150000619521,@Seductivpancake Right! Those guys have a vict...,http://a1.twimg.com/profile_images/1241412831/...,2011-10-06,5,4,2011-10-06 05:04:52,N;,nerdsherpa,95067344,en,Seductivpancake,19695580,&lt;a href=&quot;http://www.echofon.com/&quot;...
3,121813150701072385,RT @bembel &quot;Occupy Wall Street&quot; als ...,http://a0.twimg.com/profile_images/1106399092/...,2011-10-06,5,4,2011-10-06 05:04:52,N;,hamudistan,35862923,en,NaN,0,&lt;a href=&quot;http://levelupstudio.com&quot...
4,121813163778899968,#ows White shirt= Brown shirt.,http://a2.twimg.com/profile_images/1568117871/...,2011-10-06,5,4,2011-10-06 05:04:56,N;,kl_knox,419580636,en,NaN,0,&lt;a href=&quot;http://twitter.com/&quot;&gt;...


b. Print out the number of tweets and unique users for each day. Tip: You can use the csv.reader of the csv package. Some necessary replacements and processing need to be done on each line of string. At the same time, in order to relieve memory pressure, it is necessary to read and process in batches (for example, reading 100,000 lines at a time), and then read the next batch after storing and updating the results.

In [5]:
chunkSize = 10**8

# Create a dataframe reader object
chunk_reader = pd.read_csv(file_path, sep=',', quotechar='"', 
                           iterator=True, on_bad_lines='skip', chunksize=chunkSize)

# Initialize an empty DataFrame to store the group counts
group_counts = pd.DataFrame()

# Column name to group by
group_column = 'Day'  # Replace with the actual column name
id_column = 'From User ID'  # Replace with the actual column name for IDs

# Iterate over chunks
for chunk in chunk_reader:
    # Group by the specified column and count the observations in each group
    chunk_group_counts1 = chunk.groupby(group_column).size().reset_index(name='count')
    
    # Group by the specified column and calculate the number of unique IDs in each group
    chunk_group_counts2 = chunk.groupby(group_column)[id_column].nunique().reset_index(name='unique_id_count')
    
    # Append the group counts to the total group counts
    group_counts = pd.concat([group_counts, chunk_group_counts1, chunk_group_counts2], ignore_index=True)

# After processing all chunks, group again to consolidate counts across chunks
final_group_counts = group_counts.groupby(group_column).sum().reset_index()

# Sort the final group counts
final_group_counts = final_group_counts.sort_values(by='Day', ascending=True)

print(final_group_counts)

            Day    count  unique_id_count
0    2011-10-06  49638.0          18487.0
1    2011-10-07  65238.0          23460.0
2    2011-10-08  65949.0          23243.0
3    2011-10-09  65097.0          22576.0
4    2011-10-10  78619.0          27499.0
..          ...      ...              ...
131  2012-02-14  13580.0           4668.0
132  2012-02-15  13248.0           4894.0
133  2012-02-16  12837.0           4427.0
134  2012-02-17  12468.0           4299.0
135  2012-02-18   4859.0           2012.0

[136 rows x 3 columns]
